In [4]:
f_func <- "../functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))
f_path <- "../dirs.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_path, '--to=script', '--stdout'), stdout = TRUE)))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

In [2]:
Fungi_RPM = Asp_genus_RPM = Asp_fum_RPM = Fungi_count = Asp_genus_count = Asp_fum_count = total_read_count <- list()
for (conf_thr in sprintf("%1.1f",c(seq(0.0,1.0,0.1)))){

    sample_nm <- c()
    for (file in list.files(SPL_K2_REP, pattern = paste0("_conf", conf_thr, ".report"))){
        sample_nm <- c(sample_nm, unlist(strsplit(file, '_'))[1])
    }
    
    mt <- matrix(ncol = length(unique(sample_nm)), nrow = length(unique(dbs)))
    colnames(mt) <- unique(sample_nm)
    rownames(mt) <- gsub(unique(dbs), pattern = "RS_minusT2T", replacement = "RSminusT2T")
    Fungi_RPM_tmp = Asp_genus_RPM_tmp = Asp_fum_RPM_tmp = 
    root_count_tmp = Fungi_count_tmp = Asp_genus_count_tmp = Asp_fum_count_tmp = total_read_count_tmp <- mt
    

    for (file in list.files(SPL_K2_REP, pattern = paste0("_conf", conf_thr, ".report"))){
        sample <- unlist(strsplit(file, '_'))[1]
        db <- gsub(unlist(strsplit(file, '_'))[2], pattern = "RS_minusT2T", replacement = "RSminusT2T")
        if (db %nin% rownames(mt)){
            print(db); next
        }
        
        if (file.exists(paste0(SPL_STATS_DIR,sample,"_R1_04_adapt_remov_fastq.txt")) == FALSE){
            print(paste("Houston, we have a problem: ", sample))
            next
        } 
        total_read_count_tmp[db, sample] <- read.csv(file = paste0(SPL_STATS_DIR,sample,"_R1_04_adapt_remov_fastq.txt"), 
                                                 header=FALSE, sep = "\t", stringsAsFactors=FALSE)[1,1]

        k2rep <- read.csv(file = paste0(SPL_K2_REP,file), header=FALSE, sep = "\t", stringsAsFactors=FALSE)
        k2rep[, ncol(k2rep)] <- str_trim(k2rep[, ncol(k2rep)], side = "left")
        
        Fungi_count_tmp[db, sample] <- k2rep[k2rep[, ncol(k2rep)] == "Fungi", "V2"]
        Asp_genus_count_tmp[db, sample] <- k2rep[k2rep[, ncol(k2rep)] == "Aspergillus", "V2"]
        Asp_fum_count_tmp[db, sample] <- k2rep[k2rep[, ncol(k2rep)] == "Aspergillus fumigatus", "V2"]
        
        Fungi_RPM_tmp[db, sample] <- (Fungi_count_tmp[db, sample])/(total_read_count_tmp[db, sample])*10^6
        Asp_genus_RPM_tmp[db, sample] <- (Asp_genus_count_tmp[db, sample])/(total_read_count_tmp[db, sample])*10^6
        Asp_fum_RPM_tmp[db, sample] <- (Asp_fum_count_tmp[db, sample])/(total_read_count_tmp[db, sample])*10^6
        rm(k2rep)
    }
    
    Fungi_RPM[[paste0("conf", conf_thr)]] <- Fungi_RPM_tmp
    Asp_genus_RPM[[paste0("conf", conf_thr)]] <- Asp_genus_RPM_tmp
    Asp_fum_RPM[[paste0("conf", conf_thr)]] <- Asp_fum_RPM_tmp
    
    Fungi_count[[paste0("conf", conf_thr)]] <- Fungi_count_tmp
    Asp_genus_count[[paste0("conf", conf_thr)]] <- Asp_genus_count_tmp
    Asp_fum_count[[paste0("conf", conf_thr)]] <- Asp_fum_count_tmp
    total_read_count[[paste0("conf", conf_thr)]] <- total_read_count_tmp
}
saveRDS(Fungi_RPM,        paste0(INT_SPL, "Fungi_RPM.Rdata"))
saveRDS(Asp_genus_RPM,    paste0(INT_SPL, "Asp_genus_RPM.Rdata"))
saveRDS(Asp_fum_RPM,      paste0(INT_SPL, "Asp_fum_RPM.Rdata"))

saveRDS(Fungi_count,      paste0(INT_SPL, "Fungi_count.Rdata"))
saveRDS(Asp_genus_count,  paste0(INT_SPL, "Asp_genus_count.Rdata"))
saveRDS(Asp_fum_count,    paste0(INT_SPL, "Asp_fum_count.Rdata"))
saveRDS(total_read_count, paste0(INT_SPL, "total_read_count.Rdata"))

In [6]:
if (exists("Asp_species")){rm("Asp_species")}
for (db in dbs_sel_min){#unique(dbs)){
    print(db)
    for (conf_thr in sprintf("%1.1f",c(seq(0.0,1.0,0.1)))){
        print(conf_thr)
        for (file in list.files(SPL_K2_REP, pattern = paste0("_", db, "_conf", conf_thr, ".report"))){
            sample <- unlist(strsplit(file, '_'))[1]
            k2rep <- read.csv(file = paste0(SPL_K2_REP,file), header=FALSE, sep = "\t", stringsAsFactors=FALSE)
            k2rep[, ncol(k2rep)] <- str_trim(k2rep[, ncol(k2rep)], side = "left")
            
            if (file.exists(paste0(SPL_STATS_DIR,sample,"_R1_04_adapt_remov_fastq.txt")) == FALSE){
                print(paste("Houston, we have a problem: ", sample))
                next
            } 
            total_read_count <- read.csv(file = paste0(SPL_STATS_DIR,sample,"_R1_04_adapt_remov_fastq.txt"), 
                                                 header=FALSE, sep = "\t", stringsAsFactors=FALSE)[1,1]
            
            add_Asp_species <- k2rep[which(grepl(k2rep$V8,pattern = "Aspergillus") & 
                !grepl(k2rep$V8,pattern = "virus") & k2rep$V6 == "S"),] %>% 
                rename(V2 = 'count') %>% 
                mutate(RPM = count/total_read_count) %>% 
                mutate(db = db) %>% 
                mutate(threshold = conf_thr) %>% 
                mutate(taxName = V8) %>% 
                select(c("taxName", "count", "RPM", "db", "threshold")) %>% 
                mutate(sample = sample)
            if (!exists("Asp_species")){Asp_species <- add_Asp_species
                } else {Asp_species <- rbind(Asp_species, add_Asp_species)
            }
        }
    }
}
Asp_species_RPM <- Asp_species %>% select(c("taxName", "RPM", "db", "threshold", "sample"))
saveRDS(Asp_species_RPM, paste0(INT_SPL, "Asp_species_RPM.Rdata"))
        
Asp_species_count <- Asp_species %>% select(c("taxName", "count", "db", "threshold", "sample"))
saveRDS(Asp_species_count, paste0(INT_SPL, "Asp_species_count.Rdata"))

[1] "EPRSc2"
[1] "0.0"
[1] "0.1"
[1] "0.2"
[1] "0.3"
[1] "0.4"
[1] "0.5"
[1] "0.6"
[1] "0.7"
[1] "0.8"
[1] "0.9"
[1] "1.0"
[1] "EPRSFv64MCAspDM"
[1] "0.0"
[1] "0.1"
[1] "0.2"
[1] "0.3"
[1] "0.4"
[1] "0.5"
[1] "0.6"
[1] "0.7"
[1] "0.8"
[1] "0.9"
[1] "1.0"
